In [1]:
from code_tokenizer import *




def math_layer(code):
    priority = {
        '+': 1,
        '-': 1,
        '*': 2,
        '/': 2,
        '>': 3,
        '<': 3,
        '==': 3,
        '!=': 3,
        '>=': 3,
        '<=': 3,
        '&&': 4,
        '||': 5,
    }
    operations = []
    variables = []
    result = []
    # create a binary tree from the operations using the priority
    for token, value in code:
        if(token == 'symbol'):
            if(value in priority.keys()):
                current_priority = priority[value]
            else:
                current_priority = 0
            while(len(operations) > 0 and current_priority <= operations[-1][0]):
                if(len(variables) > 0 and len(result) == 0):
                    result.append(variables.pop())
                if(len(variables) > 0):
                    result.append(variables.pop())
                if(len(operations) > 0):
                    result.append(operations.pop()[1])
            operations.append([current_priority,value])
        elif(token in ['word','number']):
            variables.append(value)
    busy = True
    while(busy):
        busy = False
        if(len(variables) > 0 and len(result) == 0):
            result.append(variables.pop())
            busy = True
        if(len(variables) > 0):
            result.append(variables.pop())
            busy = True
        if(len(operations) > 0):
            result.append(operations.pop()[1])
            busy = True
    return result

code = """
my_list = [1, 2]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

# code = """
# a = 6 * 4
# b = a + 1
# """

print(color_by_char_type(code))

split = split_code(code)

def scan(value):
    i = 0
    while(i < len(value)):
        if(type(value[i]) == list):
            if(value[i][1] == '.'):
                scan(value[i+1])
                if(value[i-1][0] == 'acess_group'):
                    value[i-1][1] += value[i+1][1]
                else:
                    value[i-1] = ['acess_group',[value[i-1]]+[value[i+1]]]
                value.pop(i)
                value.pop(i)
                i -= 1
                continue
            elif(value[i][1] == '='):
                value[i-1] = ['assignment',[value[i-1]]+value[i+1:]]
                value[:] = value[:i]
                continue
            elif(value[i][1] in ['++','--']):
                value[i-1] = ['increment' if value[i][1] else 'decrement' ,value[i-1]]
                value.pop(i)
                continue
            scan(value[i])
        i += 1

scan(split)
for i in split:
    print(i)


my_list = [1, 2]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

[['assignment', [['word', 'my_list'], ['array_define', [['number', '1'], ['number', '2']]]]]]
[['assignment', [['word', 'min'], ['word', 'my_list', ['array_call', ['number', '0']]]]]]
['word', 'for', ['attached_group', [[['assignment', [['word', 'i'], ['number', '1']]]], [['word', 'i'], ['symbol', '<'], ['acess_group', [['word', 'my_list'], ['word', 'length']]]], [['increment', ['word', 'i']]]]], ['attached_block', ['word', 'if', ['attached_group', [['word', 'my_list', ['array_call', ['word', 'i']]], ['symbol', '<'], ['word', 'min']]], ['attached_block', [['assignment', [['word', 'min'], ['word', 'my_list', ['array_call', ['word', 'i']]]]]]]]]]
[['acess_group', [['word', 'console'], ['word', 'log', ['attached_group', ['word', 'min']]]]]]
